In [ ]:
import larch as lx

lx.__version__

(linear-funcs)=
# Linear Functions

In many discrete choice models, the 
probability of selecting any particular alternative is represented as
some function based on the utility of the various alternatives. 

In Larch, the utility is created based on one or more linear-in-parameters
functions, which combine raw or pre-computed data values with 
named model parameters.  To facilitate writing these functions,
Larch provides two special classes: parameter references (`P`) and
data references (`X`).

In [ ]:
from larch import P, X

Parameter and data references can be defined using either a function-like notation,
or a attribute-like notation.

In [ ]:
P("NamedParameter")

In [ ]:
# TEST
named_param = P("NamedParameter")
assert isinstance(named_param, lx.model.linear.ParameterRef)
assert named_param == "NamedParameter"

In [ ]:
X.NamedDataValue

In [ ]:
# TEST
named_data = X.NamedDataValue
assert isinstance(named_data, lx.model.linear.DataRef)
assert named_data == "NamedDataValue"

In either case, if the named value contains any spaces or non-alphanumeric characters,
it must be given in function-like notation only, as Python will not accept
those characters in the attribute-like form.

In [ ]:
P("Named Parameter")

In [ ]:
# TEST
named_param = P("Named Parameter")
assert isinstance(named_param, lx.model.linear.ParameterRef)
assert named_param == "Named Parameter"

Data references can name an exact data element that appears in the data used for 
model estimation or application, or can include simple transformations of that data, so long
as these transformations can be done without regard to any estimated parameter.
For example, we can use the log of income:

In [ ]:
X("log(INCOME)")

To write a linear-in-parameters utility function, we simply multiply together
a parameter reference and a data reference, and then optionally add that
to one or more similar terms.

In [ ]:
P.InVehTime * X.AUTO_TIME + P.Cost * X.AUTO_COST

It is permissible to omit the data reference on a term 
(in which case it is implicitly set to 1.0).

In [ ]:
P.ASC + P.InVehTime * X.AUTO_TIME + P.Cost * X.AUTO_COST

On the other hand, Larch does not currently permit you to omit the parameter 
reference from a term.  

In [ ]:
P.InVehTime * X.AUTO_TIME + P.Cost * X.AUTO_COST + X.GEN_COST

Although you cannot include a term with an implicit parameter set to 1,
you can achieve the same model structure by including that parameter *explicitly*.
Later in the model setup process you can explicitly lock any parameter to
have a specific fixed value.

In [ ]:
P.InVehTime * X.AUTO_TIME + P.Cost * X.AUTO_COST + X.GEN_COST * P.One

In [ ]:
# TEST
f = P.InVehTime * X.AUTO_TIME + P.Cost * X.AUTO_COST + X.GEN_COST * P.One
assert len(f) == 3

In addition to writing out a linear function as a single command, you can also compose
such functions over several Python commands, using both in-place and regular addition.

In [ ]:
f = P.ASC + P.InVehTime * X.AUTO_TIME
f += P.Cost * X.AUTO_COST
f

In [ ]:
f + P.Cost * X.AUTO_TOLL

Functional simplification is not automatic.  Thus, while you can subtract term from
a linear function, it does not cancel out existing terms from the function.

In [ ]:
f = P.ASC + P.InVehTime * X.AUTO_TIME
f - P.InVehTime * X.AUTO_TIME

Instead, to actually remove terms from a linear function, use the `remove_param` or `remove_data` methods.

In [ ]:
f = P.ASC + P.InVehTime * X.AUTO_TIME + P.Cost * X.AUTO_TOLL
f.remove_param(P.InVehTime)

In [ ]:
# TEST
assert len(f) == 2
assert f[0].param == "ASC"
assert f[1].param == "Cost"

In [ ]:
f = P.ASC + P.InVehTime * X.AUTO_TIME + P.Cost * X.AUTO_TOLL
f.remove_data("AUTO_TOLL")

In [ ]:
# TEST
assert len(f) == 2
assert f[0].param == "ASC"
assert f[1].param == "InVehTime"